In [2]:
import pandas as pd
import numpy as np
import pyreadr
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier,XGBRFClassifier
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix,roc_curve
from sklearn.feature_selection import SelectFromModel

In [ ]:
# read in feature_selection_params*.csv

version = 1
features = pd.read_csv(f'feature_selection_params{version}.csv'.format(version), index_col=0)
print(f'feature_selection_params{version}.csv')
#order by number of features
features = features.sort_values('number_of_features', ascending=True)
features

In [ ]:
# find the elbow point using the kneedle algorithm
from kneed import KneeLocator
#find the knee point for each feature selection method
# kneedle =KneeLocator(x=features["number_of_features"],y=features['AUCs'], S=5,curve='concave', direction='increasing', interp_method='polynomial')
kneedle = KneeLocator(x=features["number_of_features"][1:200],y=features['AUCs'][1:200],S=5,curve='concave', direction='increasing', interp_method='polynomial')
print(kneedle.knee)
# print(kneedle2.knee)

#plot the knee point
import matplotlib.pyplot as plt
plt.plot(features['number_of_features'], features['AUCs'])
plt.axvline(kneedle.knee, color='red')
#label the knee point
plt.text(kneedle.knee+10, 0.85, f"knee = {kneedle.knee}")
# plt.axvline(kneedle2.knee, color='green')
plt.xlabel('Number of features')
plt.ylabel('AUC')
plt.title(f"Feature selection method {version} AUC vs number of features")
plt.show()


In [ ]:
def cross_validate_xgboost_model( X_train, y_train, n_splits=10, suppress_output=False):
    params = {"objective": "binary:logistic", 
                "eval_metric": "auc", 
                "eta":0.1, 
                "max_depth":20,
                "lambda": 0.0003, "alpha": 0.0003, "nthread" :10}
    model = XGBClassifier(**params) 


    kf = KFold(n_splits=n_splits, random_state=6, shuffle=True)
    
    #store the accuracy and auc for each fold
    accuracy_list = []
    roc_auc_list = []
    recall_list = []
    specificity_list = []
    best_model = None

    for fold, (train_index, valid_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
        model.fit(X_train_fold, y_train_fold.values)
        y_pred = model.predict(X_valid_fold)
        y_pred_proba = model.predict_proba(X_valid_fold)[:,1]
        accuracy = accuracy_score(y_valid_fold, y_pred)
        roc_auc = roc_auc_score(y_valid_fold, y_pred_proba)
        f1 = f1_score(y_valid_fold, y_pred)
        precision = precision_score(y_valid_fold, y_pred)
        
        tn, fp, fn, tp = confusion_matrix(y_valid_fold, y_pred).ravel()
        specificity = tn / (tn+fp)

        recall = recall_score(y_valid_fold, y_pred)


        if roc_auc > max(roc_auc_list, default=0):
            best_model = model
            fpr, tpr, thresholds = roc_curve(y_valid_fold, y_pred_proba)
        accuracy_list.append(accuracy)
        roc_auc_list.append(roc_auc)
        recall_list.append(recall)
        specificity_list.append(specificity)



        if not suppress_output:
            print(f"Fold: {fold}, Accuracy: {accuracy}, ROC AUC: {roc_auc}, F1: {f1}, Precision: {precision}, Recall: {recall}, Specificity: {specificity}")

            #draw the ROC curve
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    #add label
    plt.text(0.6, 0.2, f"AUC = {roc_auc}")
        
    return {"model": best_model, "accuracy": accuracy_list, "roc_auc": roc_auc_list, "recall": recall_list, "specificity": specificity_list, "fpr": fpr, "tpr": tpr}

In [ ]:

filename = f"Rdata/external_training_test_{version}.rda"

training_test = pyreadr.read_r(filename)
train1 = training_test["um_data"]
test1 = training_test["external_data"]


print(f"train1 shape: {train1.shape}, test1 shape: {test1.shape}")

X1_train=train1.drop(["Row.names","ALS_status"],axis=1)
y1_train=train1["ALS_status"]
y1_train=y1_train.replace({'case':1,'control':0})

X1_test=test1.drop(["Row.names","ALS_status"],axis=1)
y1_test=test1["ALS_status"]
#count how many case and control in ALS_status
y1_test=y1_test.replace({'case':1,'control':0})





whole_results = cross_validate_xgboost_model(X1_train, y1_train, n_splits=10)
best_model_whole = whole_results["model"]
#print the average accuracy and roc_auc
print(f"Average accuracy: {sum(whole_results['accuracy'])/len(whole_results['accuracy'])}")


In [ ]:

thresholds = sort(best_model_whole.feature_importances_)
thresh= thresholds[-kneedle.knee]
# thresh2 = thresholds[-kneedle2.knee]




In [ ]:
# select features using threshold
selection = SelectFromModel(estimator=best_model_whole, threshold=thresh, prefit=True)
col_index = X1_train.columns[selection.get_support()]
select_X_train = selection.transform(X1_train)
select_X_train = pd.DataFrame(select_X_train, columns=col_index)
print(f"thresh: {thresh}, n={select_X_train.shape[1]}")
# train model
results = cross_validate_xgboost_model(select_X_train, y1_train, n_splits=10, suppress_output=True)
#get the average of accuracy and auc
accuracy = sum(results['accuracy'])/len(results['accuracy'])
auc = sum(results['roc_auc'])/len(results['roc_auc'])
sensitivity = sum(results['recall'])/len(results['recall'])
specificity = sum(results['specificity'])/len(results['specificity'])

fpr, tpr = results['fpr'], results['tpr']

print(f"accuracy: {accuracy*100:.2f}%, sensitivity: {sensitivity*100:.2f}%, specificity: {specificity*100:.2f}%, AUC: {auc*100:.2f}%")
print("--------------------------------------------------")

#save the model
import joblib
joblib.dump(results['model'], f"model_{version}.joblib")
features = results['model'].get_booster().feature_names
# Save selected features to txt file
with open(f"model_{version}_features.txt", "w") as f:
    for feature in features:
        f.write(f"{feature}\n")


In [139]:
def predict_model(results, X_test, y_test):
    model = results["model"]

    selected_features = model.get_booster().feature_names
    test_ds = X_test[selected_features]
    print(f"test_ds shape: {test_ds.shape}")

    y_pred = model.predict(test_ds)
    print(y_pred)
    y_pred_proba = model.predict_proba(test_ds)[:,1]

    #draw the roc curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.plot(fpr, tpr, label="external test")
    plt.plot(results["fpr"], results["tpr"], label="internal CV")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    #add label
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    #show the AUC by label
    plt.legend([f"external AUC: {roc_auc_score(y_test, y_pred_proba)*100:.2f}%", f"internal CV AUC: {sum(results['roc_auc'])/len(results['roc_auc'])*100:.2f}%"])  

    plt.show()

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    #return in percentage keep 2 decimal
    return {"accuracy": accuracy*100, "roc_auc": roc_auc*100, "f1": f1*100, "precision": precision*100, "recall": recall*100, "specificity": specificity*100}



In [ ]:
external_results = predict_model(results, X1_test, y1_test)
print(f"External test accuracy: {external_results['accuracy']:2f}%, roc_auc: {external_results['roc_auc']:.2f}%, sensitivity: {external_results['recall']:.2f}%, specificity: {external_results['specificity']:.2f}%, auc: {external_results['roc_auc']:.2f}%")

In [ ]:

feature_importance = results["model"].get_booster().get_score(importance_type='weight')
feature_importance = pd.DataFrame(feature_importance.items(), columns=['feature', 'weight'])

#merge the feature importance with the gene symbol
feature_importance 